In [1]:
import numpy as np
from numpy.random import Generator, PCG64
import timeit
import plotly.graph_objects as go
#np.random.seed(1234)

In [2]:
def exponential_rng(lam=1.0):  
    """ Generates exponential random number.
    
    Keywords:
        lam (float): the rate parameter, the inverse expectation of the distribution.
    
    Returns:
        exponential random number with rate lam.
    """
    return -np.log(np.random.rand()) / lam

In [3]:
def homogeneous_poisson_process(lam, T):
    arrivals = []
    t = exponential_rng(lam)
    while t <= T:
        arrivals.append(t)
        t += exponential_rng(lam)
    return arrivals

In [4]:
def totalRevenue(choices):
    
    revenueList = [0,1000,900,850,750,700,650,600,500,350]
    revenueList = np.flip(revenueList)
    revenue = 0
    for choice in choices:
        revenue += revenueList[choice]
        
    return revenue

In [5]:
def seatsSold(choices):
    return len(choices) - np.sum(np.array(choices) == 9)

In [6]:
lam_business = lambda t: 1.2*np.sin(t*np.pi/180.0)
lam_max_business = 1.2
lam_leisure = lambda t: 0.6*t/179
lam_max_leisure = 0.6
lam_economy = lambda t: 0.8*(1 + np.sin(t*np.pi/180.0 + np.pi))
lam_max_economy = 0.8

def non_homogeneous_poisson_process(lam_t, lam_max, T):
    arrivals = []
    t = exponential_rng(lam_max)
    while t <= T:
        if np.random.rand() < lam_t(t)/lam_max:
            arrivals.append(t)
        t += exponential_rng(lam_max)
    return arrivals

In [7]:
class Passenger:
    """ Generic event.
    
    Attributes:
        time (float): Event time.
        preferences (array): fare product preference weights.
    """
    
    def __init__(self, time):
        self.time = time
        self.preferences = np.zeros(10)
        
    def makeChoice(self,availabilities):
        
        probabilities = self.preferences*(availabilities > 0)
        probabilities = probabilities/np.sum(probabilities)
        
        cumsum = np.cumsum(probabilities)
        
        r = np.random.rand()
        
        choice = np.min(np.where(r < cumsum ))
        
        return choice
    
    def makeIdealChoice(self):
        
        probabilities = self.preferences
        probabilities = probabilities/np.sum(probabilities)
        
        cumsum = np.cumsum(probabilities)
        
        r = np.random.rand()
        
        choice = np.min(np.where(r < cumsum ))
        
        return choice
        
        
        
class Business(Passenger):

    def __init__(self, time):
        super().__init__(time)
        self.preferences = np.array([11, 15, 18, 20, 19, 15, 12, 11, 13, 7])
        
    def makeChoice(self,availabilities):
        return Passenger.makeChoice(self,availabilities)
    
    def makeIdealChoice(self):
        return Passenger.makeIdealChoice(self)
        
class Leisure(Passenger):

    def __init__(self, time):
        super().__init__(time)
        self.preferences = np.array([8, 9, 11, 12, 14, 15, 16, 18, 20, 7])
        
    def makeChoice(self,availabilities):
        return Passenger.makeChoice(self,availabilities)
    
    def makeIdealChoice(self):
        return Passenger.makeIdealChoice(self)
        
class Economy(Passenger):

    def __init__(self, time):
        super().__init__(time)
        self.preferences = np.array([1, 5, 8, 10, 11, 12, 13, 15, 20, 7])
        
    def makeChoice(self,availabilities):
        return Passenger.makeChoice(self,availabilities)
    
    def makeIdealChoice(self):
        return Passenger.makeIdealChoice(self)

In [8]:
class Scenario:
    """ Road scenario
    
    Attributes:
        planning_horizon (int): days.
        t (int): early sales discount
        lam_business (float):
        lam_leisure (float):
        lam_economy (float):
    """
    
    def __init__(self, 
                 planning_horizon=180,
                 t=21,
                 lam_business=1.2,
                 lam_leisure=0.6,
                 lam_economy=0.8,
                 flag=0, # 1 if we want to offer 1 product at a time 
                 availabilities=20*np.ones([10]),
                 cutoffs = np.zeros([10])
                ):
        
        self.planning_horizon = planning_horizon
        self.t = t
        
        self.lam_business = lam_business
        self.lam_leisure = lam_leisure
        self.lam_economy = lam_economy
        
        self.flag = flag
        
        self.availabilities = availabilities
        self.availabilities[-1] = np.Inf
        
        self.fare_offered = -1
        
        self.cutoffs = cutoffs
        
    def update_availabilities(self,choice,t):
        
        self.availabilities[choice] -= 1
        
        if self.flag:
            if self.availabilities[self.fare_offered] == 0 and self.fare_offered !=0:
                self.fare_offered -= 1
                availabilities[self.fare_offered] = 20
                
        for i, cutoff in enumerate(self.cutoffs):
        
            if t < cutoff:
                self.availabilities[i] = 0

                if self.fare_offered == i:
                    self.fare_offered -= 1

                    if self.flag:
                        self.availabilities[self.fare_offered] = 20
        
        

In [9]:
def simulate(scenario):
    """ Implements the simulation procedure.
    
    Keywords:
        scenario (Scenario): Road scenario.
    
    Returns:
        times (list): Event times. 
        queues (list): Queue length over time. 
    """
    
    #Initialise the simulation
    t = 179
    events = []
            
    
    revenues = 0
    
    #Trigger the first events
    
    # Generate customer arrivals
    business_arrival_process = non_homogeneous_poisson_process(lam_business, lam_max_business, t)
    for i in business_arrival_process:
        events.append(Business(i))
    
    leisure_arrival_process = non_homogeneous_poisson_process(lam_leisure, lam_max_leisure, t)
    for i in leisure_arrival_process:
        events.append(Leisure(i))
    
    economy_arrival_process = non_homogeneous_poisson_process(lam_economy, lam_max_economy, t)
    for i in economy_arrival_process:
        events.append(Economy(i))
        
    events.sort(key=lambda event: event.time)
    
    '''
    termination = Termination(scenario.demand_duration)
    events = [generation, termination]
    '''
    choices = []
    idealChoices = []
    
    #Main loop
    while len(events):
        
        t = events[0].time
        
        e = events[0]
        
        choice = e.makeChoice(scenario.availabilities)
        choices.append(choice)
        
        idealChoice = e.makeIdealChoice()
        idealChoices.append(idealChoice)
        
        scenario.update_availabilities(choice,t)
        
            
        events.pop(0)
    
        
    return choices, idealChoices

In [10]:
choices,idealChoices = simulate(Scenario(1))

In [56]:
def evaluate(startAvail,startCutoffs, numIter = 10):
    
    revList = []
    
    for i in range(numIter):
    
        scenario = Scenario(availabilities = startAvail,cutoffs = startCutoffs)
        choices, idealChoices = simulate(scenario)
        revenue = totalRevenue(choices)
        
        revList.append(revenue)
        
    revenue = np.mean(revList)
    std = np.std(revList)
        

    return [revenue, std]

In [57]:
def generateAvailabilities(rg,availabilities,mode='ls'):
        
    numBuckets = len(availabilities)-1
    
    buckets = np.arange(numBuckets)
    
    canChange = False
    
    if mode == 'ls':
        
        while not canChange:
        
            numToChange = rg.integers(0,high=numBuckets/2,endpoint=False)
            
            whichToChange = rg.choice(buckets,size=numToChange,replace=False)

            notYetChanged = np.setdiff1d(buckets,whichToChange)
                        
            canBeChanged = notYetChanged[np.where(availabilities[notYetChanged]>0)]
            

            if len(canBeChanged) > numToChange:
                canChange = True
        
        othersToChange = rg.choice(canBeChanged,size=numToChange,replace=False)
        
        availabilities[whichToChange] += 1
        availabilities[othersToChange] -= 1
        
    return availabilities
        
        

In [58]:
def generateCutoffs(rg,cutoffs,mode='ls'):
    
    numBuckets = len(cutoffs)-1
    
    buckets = np.arange(numBuckets)
    
    if mode == 'ls':
        
        numToChange = rg.integers(0,high=numBuckets,endpoint=True)
        whichToChange = rg.choice(buckets,size=numToChange,replace=False)
        
        for idx in whichToChange:
            if cutoffs[idx] == 0:
                cutoffs[idx]+=1
            elif rg.random() > .50:
                cutoffs[idx] += 1
            else:
                cutoffs[idx] -= 1
                
    return cutoffs
                

In [59]:
def generateSolutions(rg,solution,mode='ls',version='combined'):
    
    availabilities = solution[0]
    cutoffs = solution[1]
    
    cf = cutoffs
    av = availabilities
    
    if version == "av" or version=="combined":
    
        av = generateAvailabilities(rg,availabilities,mode)
        
    if version == 'cf' or version == 'combined':
    
        cf = generateCutoffs(rg,cutoffs,mode)
    
    return [av, cf]

In [123]:
def dominates(test,testScore,pList,listScores):
    
    print(listScores)
        
    dominatesList = []
    
    meanRev = testScore[0]
    stdRev = testScore[1]
    
    for idx, val in enumerate(listScores):
        if meanRev >= val[0] and stdRev <= val[1]:
            if meanRev > val[0] or stdRev < val[1]:
                dominatesList.append(idx)
                    
    return dominatesList
    

In [124]:
def searchPareto(rg, candidates,candidateScores):
        
    
    
    choiceIdx = rg.choice(np.arange(0,len(candidates)))
    choice = candidates[choiceIdx]
    scores = candidateScores[choiceIdx]
                          
    newCand = generateSolutions(rg,choice)
    
    newCandScore = evaluate(newCand[0],newCand[1])
         
                          
    D = []
    S = []
                          
        
    D = dominates(newCand,newCandScore,candidates,candidateScores)
    
    for i in range(len(candidates)):
                
        
        if len(dominates(candidates[i],candidateScores[i],newCand,newCandScore))>0:
            S.append(i)
            
    if len(S==0):
        solutionList = np.union1d(candidates,newCand)
        valuesList = np.union1d(candidateScores,newCandScore)
        
        for i in D:
            solutionList.pop(i)
            valuesList.pop(i)
            
            
    return solutionList, valuesList
                        

In [125]:
def LocalSearchPareto(rg, startSoln, iterations,version='combined'):
    tracker = [] # a list of tuple to track the max revenue
    
    av = startSoln[0].copy()
    cf = startSoln[1].copy()

    revenue, std = evaluate(av,cf) # calculate the inital objective function
    
    best_val = [[revenue, std]] # save the current solution as the "best solution"
    best_solution = [startSoln]                        # save the current solution as the "best solution"
    
    
    print("running algorithm...")
    start_time = timeit.default_timer() # implement a timer to track running time
    
    for i in range(1, iterations):   
        
        best_solution, best_val = searchPareto(rg,best_solution,best_val)
        
                 
                
    print("algorithm end.")
    end_time = timeit.default_timer()
    print("time taken :{0:.5f}s".format(end_time - start_time))

    
    return best_solution, most_revenue

In [126]:
rg = Generator(PCG64(4209)) # set your own unique seed number

startAv = 20*np.ones([10])
startAv[-1] = np.Inf

startCf = np.zeros([10])
startCf[3] = 21
startCf[7] = 21

startSoln = [startAv,startCf]

best_soln, best_revenue = LocalSearchPareto(rg,startSoln,100,version='av')
#best_soln, best_revenue = LocalSearch(rg,best_soln,1000,version='cf')



running algorithm...
[[9500.0, 28500.0]]
[9580.0, 28740.0]


IndexError: invalid index to scalar variable.